## LOAD FUNCTIONS AND VARIABLES

In [ ]:
import os
import math
import json
import timeit

import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import numpy as np
import itertools as it

from statistics import mean 
from Bio import SeqIO, Seq
from itertools import repeat

from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec

#------------------------------------------
cmap1=LinearSegmentedColormap.from_list("my_colormap", sns.color_palette("colorblind", n_colors=5))


sns.set_style("ticks",{'axes.grid' : True})
sns.set_palette("colorblind")

plt.rcParams["axes.linewidth"] = 1.5
plt.rcParams["xtick.major.width"] = 1.5
plt.rcParams["ytick.major.width"] = 1.5
plt.rcParams["xtick.major.size"] = 8
plt.rcParams["ytick.major.size"] = 8
plt.rcParams["axes.titlepad"] = 20

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams["axes.titlesize"] = 30
plt.rcParams['axes.labelsize'] = 23.5
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['legend.fontsize'] = 18
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Liberation Sans']
plt.rcParams['text.usetex'] = False

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams["savefig.dpi"]=300

In [ ]:
input_genomes=snakemake.params.genome
input_sample=snakemake.params.sample
input_control=snakemake.params.control
input_workdir=snakemake.params.workdir
input_figdir=snakemake.params.figdir
input_tombodir=snakemake.params.tombo_dir
input_threshold=snakemake.params.threshold_modfrac

output_modfrac_png=snakemake.output.modfrac_png
output_modfrac_kmers=snakemake.output.modfrac_kmers_table
output_coverage_png=snakemake.output.coverage_png

output_dinucleotide_histogram_pdf=snakemake.output.dinucleotide
output_trinucleotide_histogram_pdf=snakemake.output.trinucleotide
output_tetranucleotide_histogram_pdf=snakemake.output.tetranucleotide
output_pentanucleotide_histogram_pdf=snakemake.output.pentanucleotide
# output_hexanucleotide_histogram_pdf=snakemake.output.hexanucleotide


In [ ]:
threshold=input_threshold
genome=input_genomes
sample=input_sample
control=input_control
workdir=input_workdir
tombo_dir=input_tombodir + "/"

%cd $workdir

In [ ]:
def expand_list(nums):
    result = []
    for num in nums:
        expanded_range = list(range(num - 2, num + 3))  # Creates a list from num-2 to num+2
        result.extend(expanded_range)
    return sorted(set(result))  # Sorts the list and removes duplicates
        
def read_genome(fasta_file, strand):
    fasta_sequences = SeqIO.parse(open(fasta_file),'fasta')
    for fasta in fasta_sequences:
        name, sequence = fasta.id, str(fasta.seq)

    if strand=="minus":
        sequence=Seq.reverse_complement(sequence)
        
    mono=[]
    di=[]
    tri=[]
    tetra=[]
    penta=[]
    hexa=[]
    hepta=[]
    octa=[]
    nona=[]
    deca=[]

    lines=sequence + "X"*9
    for i in range(len(lines)-9):
        mono.append(lines[i])
        di.append(lines[i]+ lines[i+1])
        tri.append(lines[i]+ lines[i+1] + lines[i+2])
        tetra.append(lines[i]+ lines[i+1] + lines[i+2] + lines[i+3])
        penta.append(lines[i]+ lines[i+1] + lines[i+2] + lines[i+3] + lines[i+4])
        hexa.append(lines[i]+ lines[i+1] + lines[i+2] + lines[i+3] + lines[i+4] + lines[i+5])
        hepta.append(lines[i]+ lines[i+1] + lines[i+2] + lines[i+3] + lines[i+4] + lines[i+5] + lines[i+6])
        octa.append(lines[i]+ lines[i+1] + lines[i+2] + lines[i+3] + lines[i+4] + lines[i+5] + lines[i+6] + lines[i+7])
        nona.append(lines[i]+ lines[i+1] + lines[i+2] + lines[i+3] + lines[i+4] + lines[i+5] + lines[i+6] + lines[i+7] + lines[i+8])
        deca.append(lines[i]+ lines[i+1] + lines[i+2] + lines[i+3] + lines[i+4] + lines[i+5] + lines[i+6] + lines[i+7] + lines[i+8] + lines[i+9])

    d = {'Mononucleotide':mono,'Dinucleotide':di,'Trinucleotide':tri,'Tetranucleotide':tetra,'Pentanucleotide':penta, "Hexanucleotide":hexa, "Heptanucleotide":hepta, "Octanucleotide":octa, "Nonanucleotide":nona, "Decanucleotide":deca}

    nucleotides_df = pd.DataFrame(d)
    if strand=="plus":
        nucleotides_df['Position']=list(range(0,len(lines)-9))
    if strand=="minus":
        nucleotides_df['Position']=list(reversed(range(0,len(lines)-9)))
    return nucleotides_df

def read_modfrac(wig_file):
    greater_thans=[]
    sizes=[]
    means=[]
    position=0
    greater_than=0
    values=""

    n=0
    #WIG
    with open(wig_file) as fp:
        values=[]
        positions=[]
        greater_than=0
        for line in fp:
            n=n+1
            if n > 2:
                values.append(float(line.split()[1]))
                positions.append(float(line.split()[0]))
                if float(line.split()[1]) > threshold:
                    greater_than=greater_than+1
    modfrac_df=pd.DataFrame()
    modfrac_df["position"]=positions
    modfrac_df["Mod_fraction"]=values
    return modfrac_df


def read_valid_coverage(file_coverage):
    #Coverage
    coverage_value=[]
    coverage_position=[]

    #VALID COVERAGE
    with open(file_coverage) as fp:
        for line in fp:
            if not line.startswith("track") and not line.startswith("variable"):
                pos, cov=line.split()
                cov=int(cov)
                pos=int(pos)
                coverage_value.append(cov)
                coverage_position.append(pos)

    coverage_df=pd.DataFrame()
    coverage_df["position"]=coverage_position
    coverage_df["valid_coverage"]=coverage_value
    return coverage_df

def read_coverage_wig(wigfile):
    coverage=[1]
    mean_coverages_sample=[]
    genomes_cov_sample=[]
    values=""
    with open(wigfile) as fp:
        for line in fp:
            if (not line.startswith("track")) & (not line.startswith("variableStep")) :
                genome, low, high, cov=line.split("\t")
                if not genome in genomes_cov_sample:
                    genomes_cov_sample.append(genome)
                    mean_coverages_sample.append(mean((coverage)))
                    coverage=[]
                coverage.extend(repeat(int(cov), int(high)-int(low)))
    coverage_df=pd.DataFrame()
    coverage_df["coverage"]=coverage
    coverage_df["position"]=list(range(0,len(coverage)))

    return coverage_df

def read_modfrac_genome_denovo(genome,barcode, tombo_dir, strand):
    fasta_file="GENOMES/" + genome + ".fasta"

      
    wig_file= tombo_dir +  genome + "_" + barcode + "_" + mapping +  ".tombo_denovo/"+ genome + "_" + barcode + "_denovo.dampened_fraction_modified_reads." + strand + ".wig"
    file_valid_coverage= tombo_dir + genome + "_" + sample + "_" + control + "_" + mapping + ".tombo_denovo/" + genome + "_" + sample + "_" + control + "_" + mapping +  ".valid_coverage." + strand + ".wig"
    file_coverage_sample= tombo_dir + genome + "_" + sample + "_" + control + "_" + mapping + ".tombo_denovo/" + genome + "_" + sample + "_" + control + "_" + mapping +  ".coverage.sample." + strand + ".bedgraph"

    nucleotides_df=read_genome(fasta_file, strand)
    modfrac_df=read_modfrac(wig_file)
    valid_coverage_df=read_valid_coverage(file_valid_coverage)
    coverage_sample_df=read_coverage_wig(file_coverage_sample)
    coverage_sample_df = coverage_sample_df.rename(columns={'coverage': 'coverage_sample'})

    df=nucleotides_df.merge(modfrac_df, left_on="Position", right_on="position", how="left").merge(valid_coverage_df, left_on="Position", right_on="position", how="outer").merge(coverage_sample_df, left_on="Position", right_on="position", how="outer")
    df=df.drop("position", axis=1)
    df["strand"]=strand
    return df

def read_modfrac_genome_sampleCompare(genome,sample,control,tombo_dir, strand):
    fasta_file="GENOMES/" + genome + ".fasta"
    wig_file= tombo_dir + genome + "_" + sample + "_" + control + "_" + mapping + ".tombo_sampleCompare/" + genome + "_" + sample + "_" + control +  ".dampened_fraction_modified_reads." + strand + ".wig"
    file_valid_coverage= tombo_dir + genome + "_" + sample + "_" + control + "_" + mapping + ".tombo_sampleCompare/" + genome + "_" + sample + "_" + control + "_" + mapping +  ".valid_coverage." + strand + ".wig"
    file_coverage_sample= tombo_dir + genome + "_" + sample + "_" + control + "_" + mapping + ".tombo_sampleCompare/" + genome + "_" + sample + "_" + control + "_" + mapping +  ".coverage.sample." + strand + ".bedgraph"
    file_coverage_control= tombo_dir + genome + "_" + sample + "_" + control + "_" + mapping + ".tombo_sampleCompare/" + genome + "_" + sample + "_" + control + "_" + mapping +  ".coverage.control." + strand + ".bedgraph"

    nucleotides_df=read_genome(fasta_file, strand)
    modfrac_df=read_modfrac(wig_file)
    valid_coverage_df=read_valid_coverage(file_valid_coverage)
    coverage_sample_df=read_coverage_wig(file_coverage_sample)
    coverage_control_df=read_coverage_wig(file_coverage_control)

    coverage_sample_df =coverage_sample_df.rename(columns={'coverage': 'coverage_sample'})
    coverage_control_df =coverage_control_df.rename(columns={'coverage': 'coverage_control'})

    df=nucleotides_df.merge(modfrac_df, left_on="Position", right_on="position", how="left").drop(["position"], axis=1).merge(valid_coverage_df, left_on="Position", right_on="position", how="outer").drop(["position"], axis=1).merge(coverage_sample_df, left_on="Position", right_on="position", how="outer").drop(["position"], axis=1).merge(coverage_control_df, left_on="Position", right_on="position", how="outer")
    df=df.drop("position", axis=1)
    df["strand"]=strand
    return df


## BASIC STATISTICS

In [ ]:
dataframe_sampleCompare_plus=read_modfrac_genome_sampleCompare(genome, sample, control,tombo_dir, "plus")
dataframe_sampleCompare_minus=read_modfrac_genome_sampleCompare(genome, sample, control,tombo_dir, "minus")

print()
print("++++++++++++++++++ PLUS ++++++++++++++++++++")
print()
print(" SAMPLE MEAN COVERAGE + : \t","%.3f" % dataframe_sampleCompare_plus["coverage_sample"].mean())
print("CONTROL MEAN COVERAGE + : \t","%.3f" % dataframe_sampleCompare_plus["coverage_control"].mean())
print()
print("  SAMPLE MEAN MODFRAC + : \t","%.3f" % dataframe_sampleCompare_plus["Mod_fraction"].mean())
print()
print()
print("------------------ MINUS -------------------")
print()
print(" SAMPLE MEAN COVERAGE - : \t","%.3f" % dataframe_sampleCompare_minus["coverage_sample"].mean())
print("CONTROL MEAN COVERAGE - : \t","%.3f" % dataframe_sampleCompare_minus["coverage_control"].mean())
print()
print("  SAMPLE MEAN MODFRAC - : \t","%.3f" % dataframe_sampleCompare_minus["Mod_fraction"].mean())


## MODFRAC PER POSITION (FIRST 10000bp PLUS)

In [ ]:
step=500
nrows=math.ceil(len(dataframe_sampleCompare_plus)/step)
nrows=min(40,nrows)
n=0

palette ={"A": "C0", "T": "C1", "G": "C2", "C": "C3"}
fig, axs = plt.subplots(nrows=nrows, ncols=1,figsize=(40,3*nrows), sharey=True)

while n < nrows:
    min_x=n*step
    max_x=(n+1)*step
    subset=dataframe_sampleCompare_plus[(dataframe_sampleCompare_plus["Position"]>(min_x)) & (dataframe_sampleCompare_plus["Position"]<max_x)]

    filtered_df = subset[subset['Mod_fraction'] > 0.7]
    positions_filtered = filtered_df["Position"].to_list()
    expanded_positions_filtered = expand_list(positions_filtered)
    expanded_filtered_df=subset[subset["Position"].isin(expanded_positions_filtered)]

#     ax=sns.barplot(data=subset, x='Position', y='Mod_fraction', ax=axs[n], hue="Mononucleotide", dodge=False, palette=palette)

    subset_A=expanded_filtered_df[expanded_filtered_df["Mononucleotide"]=="A"]
    subset_C=expanded_filtered_df[expanded_filtered_df["Mononucleotide"]=="C"]
    subset_G=expanded_filtered_df[expanded_filtered_df["Mononucleotide"]=="G"]
    subset_T=expanded_filtered_df[expanded_filtered_df["Mononucleotide"]=="T"]



    axs[n].stem(subset_A["Position"], subset_A["Mod_fraction"], 'C0', markerfmt = 'C0o', label = "A" )
    axs[n].stem(subset_C["Position"], subset_C["Mod_fraction"], 'C3', markerfmt = 'C3o', label = "C" )
    axs[n].stem(subset_G["Position"], subset_G["Mod_fraction"], 'C2', markerfmt = 'C2o', label = "G" )
    axs[n].stem(subset_T["Position"], subset_T["Mod_fraction"], 'C1', markerfmt = 'C1o', label = "T" )



    axs[n].set_ylim(0, 1.1)
    axs[n].set_xlim(min_x, max_x)
    axs[n].legend(loc=1, fontsize=24)
    axs[n].tick_params(axis='x', which='major', labelsize=30)
    axs[n].tick_params(axis='y', which='major', labelsize=16)


    n=n+1
    print("{:10.2f}".format(100*(n)/nrows), "%",end = "\r" )


plt.tight_layout()
fig.savefig(output_modfrac_png)
plt.show()

## COVERAGE PER POSITION 

In [ ]:
step=25000
n=0
nrows=math.ceil(len(dataframe_sampleCompare_plus)/step)

palette ={"A": "C0", "T": "C1", "G": "C2", "C": "C3"}
fig, axs = plt.subplots(nrows=nrows, ncols=1,figsize=(30,8*nrows), sharey=True)

while n < nrows:
    min_x=n*step
    max_x=(n+1)*step
    subset_plus=dataframe_sampleCompare_plus[(dataframe_sampleCompare_plus["Position"]>(min_x)) & (dataframe_sampleCompare_plus["Position"]<max_x)]
    subset_minus=dataframe_sampleCompare_minus[(dataframe_sampleCompare_minus["Position"]>(min_x)) & (dataframe_sampleCompare_minus["Position"]<max_x)]

    sns.lineplot(data=subset_plus, x='Position', y='coverage_sample', ax=axs[n], label="sample +")
    sns.lineplot(data=subset_plus, x='Position', y='coverage_control', ax=axs[n], label="control +")
    sns.lineplot(data=subset_minus, x='Position', y='coverage_sample', ax=axs[n], label="sample -")
    sns.lineplot(data=subset_minus, x='Position', y='coverage_control', ax=axs[n], label="control -")
    
    axs[n].set_ylabel("coverage")
    axs[n].set_xlim(min_x, max_x)

    axs[n].legend(loc=1, fontsize=24)
    axs[n].tick_params(axis='x', which='major', labelsize=20)


    print("{:10.2f}".format(100*n/nrows), "%",end = "\r" )
    n=n+1


plt.tight_layout()
fig.savefig(output_coverage_png)
plt.show()


## KMER STATISTICS

In [ ]:
df=pd.concat([dataframe_sampleCompare_plus,dataframe_sampleCompare_minus]).reset_index()

modfrac=df["Mod_fraction"].to_list()
modfrac_p1=modfrac[1:] + modfrac[:1]
modfrac_p2=modfrac[2:] + modfrac[:2]
modfrac_p3=modfrac[3:] + modfrac[:3]
modfrac_p4=modfrac[4:] + modfrac[:4]
modfrac_p5=modfrac[5:] + modfrac[:5]
modfrac_p6=modfrac[6:] + modfrac[:6]
modfrac_p7=modfrac[7:] + modfrac[:7]
modfrac_p8=modfrac[8:] + modfrac[:8]
modfrac_p9=modfrac[9:] + modfrac[:9]
modfrac_p10=modfrac[10:] + modfrac[:10]

modfrac_m1=modfrac[-1:] + modfrac[:-1]
modfrac_m2=modfrac[-2:] + modfrac[:-2]

df["modfrac_p1"]=modfrac_p1
df["modfrac_p2"]=modfrac_p2
df["modfrac_p3"]=modfrac_p3
df["modfrac_p4"]=modfrac_p4
df["modfrac_p5"]=modfrac_p5
df["modfrac_p6"]=modfrac_p6
df["modfrac_p7"]=modfrac_p7
df["modfrac_p8"]=modfrac_p8
df["modfrac_p9"]=modfrac_p9
df["modfrac_p10"]=modfrac_p10

df["modfrac_m1"]=modfrac_m1
df["modfrac_m2"]=modfrac_m2

mono=df["Mononucleotide"].to_list()

nuc_p1=mono[1:] + mono[:1]
nuc_p2=mono[2:] + mono[:2]
nuc_p3=mono[3:] + mono[:3]
nuc_p4=mono[4:] + mono[:4]
nuc_p5=mono[5:] + mono[:5]
nuc_p6=mono[6:] + mono[:6]
nuc_p7=mono[7:] + mono[:7]
nuc_p7=mono[7:] + mono[:7]
nuc_p8=mono[8:] + mono[:8]
nuc_p9=mono[9:] + mono[:9]
nuc_p10=mono[10:] + mono[:10]

nuc_m1=mono[-1:] + mono[:-1]
nuc_m2=mono[-2:] + mono[:-2]

df["nuc_p1"]=nuc_p1
df["nuc_p2"]=nuc_p2
df["nuc_p3"]=nuc_p3
df["nuc_p4"]=nuc_p4
df["nuc_p5"]=nuc_p5
df["nuc_p6"]=nuc_p6
df["nuc_p7"]=nuc_p7
df["nuc_p7"]=nuc_p7
df["nuc_p8"]=nuc_p8
df["nuc_p9"]=nuc_p9
df["nuc_p10"]=nuc_p10

df["nuc_m1"]=nuc_m1
df["nuc_m2"]=nuc_m2
df.columns

df[['Position','Mononucleotide', 'Dinucleotide', 'Trinucleotide', 'Tetranucleotide',
       'Pentanucleotide', 'Hexanucleotide', 'Heptanucleotide', 'Octanucleotide', 'Nonanucleotide', 'Decanucleotide', 'coverage_sample', 'coverage_control', "modfrac_m2", "modfrac_m1", 'Mod_fraction', 'modfrac_p1', 'modfrac_p2', 'modfrac_p3', 'modfrac_p4',
       'modfrac_p5', 'modfrac_p6', 'modfrac_p7', 'modfrac_p8', 'modfrac_p9',
       'modfrac_p10']].set_index("Position").to_csv(output_modfrac_kmers)


In [ ]:
categories=["Mononucleotide","Dinucleotide","Trinucleotide","Tetranucleotide",
             "Pentanucleotide","Hexanucleotide","Heptanucleotide",
             "Octanucleotide", "Nonanucleotide", "Decanucleotide"]
modfracs=['coverage_sample', 'coverage_control', "modfrac_m2","modfrac_m1","Mod_fraction","modfrac_p1", "modfrac_p2",
          "modfrac_p3","modfrac_p4","modfrac_p5","modfrac_p6",
          "modfrac_p7","modfrac_p8", "modfrac_p9","modfrac_p10"]
times=[]
n=0
cmap="rainbow"
for category in categories:
    start = timeit.default_timer()
    sel_cols=modfracs[0:7+n]
    sel_cols.append(category)
    df_nucleotide=df[sel_cols].groupby(category).mean()
    df_count = df[sel_cols].groupby(category).size().rename('count').to_frame()
    df_nucleotide["max"]= df_nucleotide[modfracs[2:7+n]].T.max()
    df_nucleotide = df_count.join(df_nucleotide)    
    df_nucleotide[(~df_nucleotide.index.str.contains("X")) & (df_nucleotide["max"]>0.3)].sort_values(by="max", ascending=False).style.background_gradient(cmap=cmap,vmin=0.15, vmax=1).to_html(input_figdir + "/" + genome + "/sampleCompare_" + mapping + "/sampleCompare_" + genome + "_" + sample + "_" + control + "_" + mapping +"_table_"+ category + ".html")

    print( category + ":  ", timeit.default_timer() - start) 
    times.append(timeit.default_timer() - start)
    n=n+1


## DINUCLEOTIDE

In [ ]:
import itertools as it
combinations_temp=list(it.product(["A", "C", "G", "T"], repeat=2))
combinations=[]
for comb in combinations_temp:
    combinations.append(''.join(comb))
combinations


df = df[~df["Dinucleotide"].str.contains("X", na=False)]

ncols=4
nrows=int(np.ceil(len(combinations)/ncols))
fig, axs = plt.subplots(nrows=nrows, ncols=ncols ,figsize=(8,2*nrows), sharey=True)
n=0
i=0
j=0
for kmer in combinations:
    name=kmer
    group=df[df["Dinucleotide"] == kmer]
    if len(group)>0:
        x=-2
        j=int(np.floor((n)/ncols))
        i=n-(j*ncols)

        for column in ["modfrac_m2", "modfrac_m1", "Mod_fraction", "modfrac_p1",  "modfrac_p2", "modfrac_p3"]:
            if x==0 or x==1:
                color=palette[name[x]]
            else:
                color="gray"
            data=group[column].to_list()
            mean=np.nanmean(data)
            std_error = np.nanstd(data, ddof=1)
            ax=axs[j,i]
            ax.bar(x=x, #x-coordinates of bars
               height=mean, #height of bars
               yerr=std_error, #error bar width
               width=1,
               capsize=4, color=color) #length of error bar caps
            x=x+1
            ax.set_ylim(0, 1)
            ax.set_xticks(range(-2, 4, 1))
            ax.set_xticklabels(["N","N",name[0],name[1],"N","N"], )
            ax.tick_params(axis='x', which='major', labelsize=16)
        ax.set_title(name,x=0.5, y=0.60, fontsize=16)

    n=n+1
    print("{:10.2f}".format(100*n/len(combinations)), "%",end = "\r" )

plt.tight_layout()
plt.savefig(output_dinucleotide_histogram_pdf, format="pdf", bbox_inches="tight")
plt.show()

## TRINUCLEOTIDE

In [ ]:
import itertools as it
combinations_temp=list(it.product(["A", "C", "G", "T"], repeat=3))
combinations=[]
for comb in combinations_temp:
    combinations.append(''.join(comb))
combinations

df = df[~df["Trinucleotide"].str.contains("X", na=False)]


ncols=8
nrows=int(np.ceil(len(combinations)/ncols))
fig, axs = plt.subplots(nrows=nrows, ncols=ncols ,figsize=(14,2*nrows), sharey=True)
n=0
i=0
j=0
for kmer in combinations:
    name=kmer
    group=df[df["Trinucleotide"] == kmer]
    
    x=-2
    j=int(np.floor((n)/ncols))
    i=n-(j*ncols)
    
    if len(group)>0:

        for column in ["modfrac_m2", "modfrac_m1", "Mod_fraction", "modfrac_p1",  "modfrac_p2", "modfrac_p3", "modfrac_p4"]:
            if x==0 or x==1 or x==2:
                color=palette[name[x]]
            else:
                color="gray"
            data=group[column].to_list()
            mean=np.nanmean(data)
            std_error = np.nanstd(data, ddof=1)
            ax=axs[j,i]
            ax.bar(x=x, #x-coordinates of bars
               height=mean, #height of bars
               yerr=std_error, #error bar width
               width=.9,
               capsize=4, color=color) #length of error bar caps
            x=x+1
            ax.set_ylim(0, 1)
            ax.set_xticks(range(-2, 5, 1))
            ax.set_xticklabels(["N","N",name[0],name[1],name[2],"N","N"], )
            ax.tick_params(axis='x', which='major', labelsize=14)
            ax.set_title(name,x=0.5, y=0.60, fontsize=18)
    
    n=n+1
    print("{:10.2f}".format(100*n/len(combinations)), "%",end = "\r" )
plt.tight_layout()
plt.savefig(output_trinucleotide_histogram_pdf, format="pdf", bbox_inches="tight")
plt.show()

## TETRANUCLEOTIDE

In [ ]:
combinations_temp=list(it.product(["A", "C", "G", "T"], repeat=4))
combinations=[]
for comb in combinations_temp:
    combinations.append(''.join(comb))
combinations


df = df[~df["Tetranucleotide"].str.contains("X", na=False)]


ncols=16
nrows=int(np.ceil(len(combinations)/ncols))
fig, axs = plt.subplots(nrows=nrows, ncols=ncols ,figsize=(30,2*nrows), sharey=True)
n=0
i=0
j=0
min_modfrac=0.7

for kmer in combinations:
    name=kmer
    group=df[df["Tetranucleotide"] == kmer]
    x=-2
    j=int(np.floor((n)/ncols))
    i=n-(j*ncols)
    
    if len(group)>0:

            #print(i,j)
            for column in ["modfrac_m2", "modfrac_m1", "Mod_fraction", "modfrac_p1",  "modfrac_p2", "modfrac_p3", "modfrac_p4","modfrac_p5"]:
                if x==0 or x==1 or x==2 or x==3:
                    color=palette[name[x]]
                else:
                    color="gray"
                data=group[column].to_list()
                mean=np.nanmean(data)
                std_error = np.nanstd(data, ddof=1)
                ax=axs[j,i]
                ax.bar(x=x, #x-coordinates of bars
                   height=mean, #height of bars
                   yerr=std_error, #error bar width
                   width=.9,
                   capsize=4, color=color) #length of error bar caps
                x=x+1
            ax.set_ylim(0, 1)
            ax.set_xticks(range(-2, 6, 1))
            ax.set_xticklabels(["N","N",name[0],name[1],name[2],name[3],"N","N"], )
            ax.tick_params(axis='x', which='major', labelsize=10)
            ax.set_title(name,x=0.5, y=0.60, fontsize=18)


    n=n+1
    print("{:10.2f}".format(100*n/len(combinations)), "%",end = "\r" )
        
plt.tight_layout()
plt.savefig(output_tetranucleotide_histogram_pdf, format="pdf", bbox_inches="tight")
plt.show()

# PENTANUCLEOTIDE

In [ ]:
combinations_temp=list(it.product(["A", "C", "G", "T"], repeat=5))
combinations=[]
for comb in combinations_temp:
    combinations.append(''.join(comb))
combinations


df = df[~df["Pentanucleotide"].str.contains("X", na=False)]

ncols=32
nrows=int(np.ceil(len(combinations)/ncols))
fig, axs = plt.subplots(nrows=nrows, ncols=ncols ,figsize=(65,2*nrows), sharey=True)
n=0
i=0
j=0

min_modfrac=0.7
for kmer in combinations:
    name=kmer
    group=df[df["Pentanucleotide"] == kmer]
    x=-2
    j=int(np.floor((n)/ncols))
    i=n-(j*ncols)
    
    if len(group)>0:

        if ((group["Mod_fraction"].mean() > min_modfrac) or (group["modfrac_p1"].mean() > min_modfrac) or (group["modfrac_p2"].mean() > min_modfrac) or (group["modfrac_p3"].mean() > min_modfrac)):
            for column in ["modfrac_m2", "modfrac_m1", "Mod_fraction", "modfrac_p1",  "modfrac_p2", "modfrac_p3", "modfrac_p4","modfrac_p5","modfrac_p6"]:
                if x==0 or x==1 or x==2 or x==3 or x==4:
                    color=palette[name[x]]
                else:
                    color="gray"
                data=group[column].to_list()
                mean=np.nanmean(data)
                std_error = np.nanstd(data, ddof=1)
                ax=axs[j,i]
                ax.bar(x=x, #x-coordinates of bars
                   height=mean, #height of bars
                   yerr=std_error, #error bar width
                   width=.9,
                   capsize=4, color=color) #length of error bar caps
                x=x+1
                ax.set_ylim(0, 1)
                ax.set_xticks(range(-2, 7, 1))
                ax.set_xticklabels(["N","N",name[0],name[1],name[2],name[3],name[4],"N","N"], )
                ax.tick_params(axis='x', which='major', labelsize=14)
                ax.set_title(name,x=0.5, y=0.60, fontsize=14)


    n=n+1
    print("{:10.2f}".format(100*n/len(combinations)), "%",end = "\r" )


plt.tight_layout()
plt.savefig(output_pentanucleotide_histogram_pdf, format="pdf", bbox_inches="tight")
plt.show()

## HEXANUCLEOTIDE

In [ ]:
# combinations_temp=list(it.product(["A", "C", "G", "T"], repeat=6))
# combinations=[]
# for comb in combinations_temp:
#     combinations.append(''.join(comb))
# combinations



# df = df[~df["Hexanucleotide"].str.contains("X", na=False)]


# ncols=64
# nrows=int(np.ceil(len(combinations)/ncols))
# fig, axs = plt.subplots(nrows=nrows, ncols=ncols ,figsize=(250,2*nrows), sharey=True)
# n=0
# i=0
# j=0
# min_modfrac=0.7
# for kmer in combinations:
#     name=kmer
#     group=df[df["Hexanucleotide"] == kmer]
#     x=-2
#     j=int(np.floor((n)/ncols))
#     i=n-(j*ncols)
    
#     if len(group)>0:

#         if ((group["Mod_fraction"].mean() > min_modfrac) or (group["modfrac_p1"].mean() > min_modfrac) or (group["modfrac_p2"].mean() > min_modfrac) or (group["modfrac_p3"].mean() > min_modfrac)):
#             for column in ["modfrac_m2", "modfrac_m1", "Mod_fraction", "modfrac_p1",  "modfrac_p2", "modfrac_p3", "modfrac_p4","modfrac_p5","modfrac_p6","modfrac_p7"]:
#                 if x==0 or x==1 or x==2 or x==3:
#                     color=palette[name[x]]
#                 else:
#                     color="gray"
#                 data=group[column].to_list()
#                 mean=np.nanmean(data)
#                 std_error = np.nanstd(data, ddof=1)
#                 ax=axs[j,i]
#                 ax.bar(x=x, #x-coordinates of bars
#                    height=mean, #height of bars
#                    yerr=std_error, #error bar width
#                    width=.9,
#                    capsize=4, color=color) #length of error bar caps
#                 x=x+1
#                 ax.set_ylim(0, 1)
#                 ax.set_xticks(range(-2, 8, 1))
#                 ax.set_xticklabels(["N","N",name[0],name[1],name[2],name[3],name[4],name[5],"N","N"], )
#                 ax.tick_params(axis='x', which='major', labelsize=10)            
#                 ax.set_title(name,x=0.5, y=0.60, fontsize=10)
#     n=n+1
#     print("{:10.2f}".format(100*n/len(combinations)), "%",end = "\r" )

# plt.tight_layout()
# # plt.savefig(output_hexanucleotide_histogram_pdf, format="pdf", bbox_inches="tight")
# plt.show()